# Number Representation and Precision

Real numbers are stored with a decimal precision (or mantissa) and the decimal exponent range. The mantissa contains the significant figures of the number (and thereby the precision of the number). A number like (9.90625)10 in the decimal representation is given in a binary representation by

(1001.11101)$_2$ = $1\times2^3 +0\times2^2 +0\times2^1 +1\times2^0 +1\times2^{−1} +1\times2^{−2} +1\times2^{−3} +0\times2^{−4} +1 \times 2^{−5}$

and it has an exact machine number representation since we need a finite number of bits to represent this number. This representation is however not very practical. Rather, we prefer to use a scientific notation. In the decimal system we would write a number like 9.90625 in what is called the normalized scientific notation. This means simply that the decimal point is shifted and appropriate powers of 10 are supplied. Our number could then be written as
$9.90625 = 0.990625 \times 10^1$,
and a real non-zero number could be generalized as
$x = \pm r \times 10^n$,
with a $r$ a number in the range $1/10 \le r < 1$. In a similar way we can represent a binary number in
scientific notation as
$x = \pm q \times 2^m$,
with a $q$ a number in the range $1/2 \le q < 1$.

In a typical computer, floating-point numbers are represented in the way described above, but with certain restrictions on q and m imposed by the available word length. In the machine, our number x is represented as

$x = (−1)^s \times mantissa \times 2^{exponent}$

where $s$ is the sign bit, and the exponent gives the available range. With a single-precision word, 32 bits, 8 bits would typically be reserved for the exponent, 1 bit for the sign and 23 for the mantissa. 

## 32-bit – single precision:

Sign bit: 1 bit

Exponent: 8 bits

Significand precision: 24 bits (23 explicitly stored)

This gives 6–9 significant decimal digits precision!

## 64-bit = double precision:

Sign bit: 1 bit

Exponent: 11 bits

Significand precision: 53 bits (52 explicitly stored)

This gives 15–17 significant decimal digits precision.
This the the Python default standard


## 128-bit = quadruple precision:

Sign bit: 1 bit

Exponent: 15 bits

Significand precision: 113 bits (112 explicitly stored)

This gives 33–36 significant decimal digits precision.


## 256-bit – Octuple precision:

Sign bit: 1 bit
    
Exponent: 19 bits
    
Significand precision: 237 bits (236 explicitly stored)

THIS IS RARELY IMPLEMENTED


# Precision effects

One important consequence of rounding error is that you should **NEVER Use an if statment to test equality of two floats.**  For instance, you should never, in any program, have a statment like:

In [ ]:
if x == 3.3:
    print(x)

If you need to do a logic trigger based on a float:

In [ ]:
epsilon = 1e-12
if abs(x-3.3) < epsilon:
    print(x)

## Which operations are most important in dealing with precision?

__Subtraction__ and __Derivatives__

## Subtraction

a = b - c

We have:   $fl(a) = fl(b) - fl(c) = a(1+\epsilon_a)$  or
            $fl(a) = b(1+\epsilon_b) - c(1+\epsilon_c)$
            
So, $fl(a)/a = 1 + \epsilon_b (b/a) - \epsilon_c (c/a)$

IF $b \sim c$, we have the potential of increased error on $fl(a)$


If we have:

$x = 1000000000000000$

$y = 1000000000000001.2345678901234$

as far the computer is concerned:
    

In [1]:
x = 1000000000000000
y = 1000000000000001.2345678901234
 
print(y-x) 

1.25


**The true result should be 1.2345678901234!**

Always avoid subtracting two numbers that are very close together.

In other words, instead of 16-figure accuracy we now only have three figures and the fractional error is a few percent of the true value.  This is much worse than before!


To see another exanple of this in practice, consider two numbers:

$x = 1$, and $ y = 1+10^{-14}\sqrt 2$ 

Simply we can see that:

$ 10^{14} (y - x) = \sqrt 2$

Let us try the same calculation in python:
 

In [2]:
from math import sqrt
x = 1.0
y = 1.0 + (1e-14)*sqrt(2)

print((1e14)*(y-x))
print(sqrt(2))

1.4210854715202004
1.4142135623730951


Again error off by a percent. (Because everything is stored as an approximation.) We need to be careful in how we code math!

## Example 1:  Summing $1/n$ 

Consider the series:

$$s_1 = \sum_{n=1}^N \frac{1}{n}$$ which is finite when N is finite, then consider

$$s_2 = \sum_{n=N}^1 \frac{1}{n}$$ which when summed analyitically should give $s_2 = s_1$

In [109]:
s1=0
for n in range(1,10**8):    
    s1+=1/n
print(s1)

# Write a code to perform both of these to sums for N = 1e8 and compare

18.997896403852554


In [124]:
s1=0
for n in range(10**8,1,-1):    
    s1+=1/n
print(s1)

17.997896413853447


In [39]:
print(18.997896403852554-17.997896413853447)

0.9999999899991074


## Example 2: $e^{-x}$

There are three possible algorithms for $e^{-x}$

1) Simple: $$e^{-x} = \sum_{n=0}^{\infty} (-1)^n \; \frac{x^n}{n!}$$  

2) Recursion: $$e^{-x} = \sum_{n=0}^{\infty} s_n = \sum_{n=0}^{\infty} (-1)^n \; \frac{x^n}{n!}$$  where $$ s_n = -s_{n-1} \frac{x}{n}$$ $s0=0$

3) Inverse:  $$e^{x} {\sum_{n=0}^{\infty} \frac{x^n}{n!}}$$  Then take the inverse:   $$e^{-x} = \frac{1}{e^{x}}$$


In [149]:
import math
import numpy as np
np.exp(-1)

def emxsmp(x):
    sum1 = 0
    for n in range(0,100,10):
        sum1 += ((-1)**n)*(int(x)**n)/(math.factorial(n))
    print(sum1)
    return emxsmp
print(emxsmp(2))

1.0002821869492846
<function emxsmp at 0x0000010F617754C0>


In [152]:
import math
import numpy as np
np.exp(-1)
def emxrec(x):
    sum2 = 0
    #when n=0, so does the rest of the phrase. Zero can \
    #therefore be excluded from the range.
    for n in range(1,100,10):
        m=n-1
        body2 = ((-1)**m)*(x**m)/(math.factorial(m))
        sum2 += (x/n)*(-body2)
    print(sum2)
    return emxrec
print(emxrec(2))



-2.000051306718014
<function emxrec at 0x0000010F61775160>


In [153]:
import numpy as np
np.exp(-1)
def emxinv(x):
    sum3 = 0
    for n in range(0,100,10):
        body3 = (x**n)/(math.factorial(n))
        sum3 += body3
    sum3b = 1/sum3
    print(sum3b)    
    return emxinv
print(emxinv(2))


0.9997178926577257
<function emxinv at 0x0000010F617755E0>


In [161]:
import numpy as np
np.exp(-1)

# write a function to compute e^-X for all three methods 
# Then chack their output for x = 0 - 100, in steps of 10 and 
# Compare to the numpy version of exp(-x) which is imported above. 

def emxsmp(x):
    sum1 = 0
    for n in range(0,100,10):
        sum1 += ((-1)**n)*(int(x)**n)/(math.factorial(n))
    print(sum1)
    return emxsmp

def emxrec(x):
    sum2 = 0
    #when n=0, so does the rest of the phrase. Zero can \
    #therefore be excluded from the range.
    for n in range(1,100,10):
        m=n-1
        body2 = ((-1)**m)*(x**m)/(math.factorial(m))
        sum2 += (x/n)*(-body2)
    print(sum2)
    return emxrec

def emxinv(x):
    sum3 = 0
    for n in range(0,100,10):
        body3 = (x**n)/(math.factorial(n))
        sum3 += body3
    sum3b = 1/sum3
    print(sum3b)    
    return emxinv

print(1, emxsmp(1), emxrec(1), emxinv(1), np.exp(-1))

1.0000002755731923
-1.0000000250521084
0.9999997244268837
1 <function emxsmp at 0x0000010F61775820> <function emxrec at 0x0000010F61775CA0> <function emxinv at 0x0000010F61775310> 0.36787944117144233
